In [1]:
import pandas as pd
import pickle
import os

In [2]:
from utils import haversine
from utils import join_data, train_test,hervy_dist, minutes_convert, haversine,taxi_demand_cluster, predict_lat_long, plt_map

In [3]:
df_model = pd.read_csv("Classical_ML_Models_test.csv")

In [4]:
df_model = pd.read_csv("Classical_ML_Models_test.csv")

In [5]:
df_model.drop("Unnamed: 0", axis=1, inplace=True)

In [6]:
df_model.sort_values(by= "Score", inplace=True)

In [7]:
filename = df_model.iloc[0]["File_name"]

In [8]:
model = pickle.load(open(filename, 'rb'))
scaler = pickle.load(open('scaler.pkl','rb'))

In [9]:
df_test  = pd.read_csv("Test_Data.csv")
y_test = pd.read_csv("Y_Test.csv")
df_test= df_test.reset_index()
y_test = y_test.reset_index()
df_test= df_test.drop(["index","Unnamed: 0"], axis=1)
y_test= y_test.drop(["index","Unnamed: 0"], axis=1)

In [10]:
df_final = pd.concat([df_test, y_test], axis=1)

In [11]:
df_final

,Latitude,Longitude,Hour,Day,Minutes_Identifier,Latitude_1,Longitude_1
0,37.78762,-122.41186,5,6,3,37.78777,-122.41206
1,37.92079,-122.31416,10,5,2,37.79251,-122.42040
2,37.73439,-122.38998,9,6,3,37.75002,-122.38753
3,37.76616,-122.41053,19,6,1,37.78235,-122.41009
4,37.77492,-122.41233,9,6,3,37.77773,-122.41982
...,...,...,...,...,...,...,...
45644,37.77576,-122.43211,9,6,3,37.78714,-122.42007
45645,37.78824,-122.41228,6,6,3,37.78593,-122.40692
45646,37.76614,-122.43111,1,2,2,37.76094,-122.43473
45647,37.77065,-122.44709,19,2,3,37.76952,-122.45067


In [12]:
df_final["Distance"]=df_final.apply(lambda x : haversine([x.Latitude, x.Longitude ], [x.Latitude_1, x.Longitude_1]), axis=1)

In [32]:
total_distance = df_final["Distance"].sum()/1000
total_distance = total_distance/1.6

In [14]:
df_test["Latitude"] = df_test["Latitude"]/90
df_test["Longitude"] = df_test["Longitude"]/180
df_test[['Hour', 'Day', 'Minutes_Identifier']] = scaler.fit_transform(df_test[['Hour', 'Day', 'Minutes_Identifier']])

In [15]:
df_pre = model.predict(df_test)

In [16]:
df_pre=pd.DataFrame(df_pre)
df_pre.columns =["Latitude_pre","Longitude_pre"]

In [17]:
df_final = pd.concat([df_final, df_pre], axis=1)

In [18]:
df_final

,Latitude,Longitude,Hour,Day,Minutes_Identifier,Latitude_1,Longitude_1,Distance,Latitude_pre,Longitude_pre
0,37.78762,-122.41186,5,6,3,37.78777,-122.41206,24.229810,37.787411,-122.411537
1,37.92079,-122.31416,10,5,2,37.79251,-122.42040,17042.914253,37.775146,-122.410042
2,37.73439,-122.38998,9,6,3,37.75002,-122.38753,1751.277358,37.759140,-122.407852
3,37.76616,-122.41053,19,6,1,37.78235,-122.41009,1800.661190,37.764759,-122.411346
4,37.77492,-122.41233,9,6,3,37.77773,-122.41982,728.682031,37.775784,-122.412689
...,...,...,...,...,...,...,...,...,...,...
45644,37.77576,-122.43211,9,6,3,37.78714,-122.42007,1649.496797,37.774273,-122.429398
45645,37.78824,-122.41228,6,6,3,37.78593,-122.40692,536.503164,37.787857,-122.411354
45646,37.76614,-122.43111,1,2,2,37.76094,-122.43473,659.993565,37.768902,-122.422958
45647,37.77065,-122.44709,19,2,3,37.76952,-122.45067,338.829663,37.770218,-122.440773


In [19]:
df_final["Distance_Pred"]=df_final.apply(lambda x : haversine([x.Latitude, x.Longitude ], [x.Latitude_pre, x.Longitude_pre]), axis=1)

In [33]:
total_pred_distance = df_final["Distance_Pred"].sum()/1000
total_pred_distance = total_pred_distance/1.6

In [34]:
1- total_pred_distance/total_distance

0.31204806667367824

In [41]:
total_pred_distance*404

19070223.694537964

In [37]:
total_distance*404

27720285.05295621